# Credit Card Case Study – Complete Analysis
Using Python, Pandas, NumPy, Matplotlib, Seaborn

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
plt.rcParams['figure.figsize']=(10,5)

acq = pd.read_csv('/mnt/data/Customer Acqusition.csv')
spend = pd.read_csv('/mnt/data/spend.csv')
repay = pd.read_csv('/mnt/data/Repayment.csv')

acq.head(), spend.head(), repay.head()


## 1. Data Cleaning Rules

In [ ]:

# Fix age <18
mean_age = acq['Age'].mean()
acq.loc[acq['Age']<18,'Age']=mean_age

# Merge spend + acquisition
data = spend.merge(acq,on='Customer',how='left')

# Rule: spend > limit → 50% of limit
data.loc[data['Amount']>data['Limit'],'Amount'] = data['Limit']*0.5

# Clean repayment > limit
repay = repay.merge(acq[['Customer','Limit']],on='Customer',how='left')
repay.loc[repay['Amount']>repay['Limit'],'Amount'] = repay['Limit']


## 2. Summary Calculations

In [ ]:

distinct_customers = acq['Customer'].nunique()
distinct_categories = spend['Category'].nunique()

spend['Month']=pd.to_datetime(spend['Date']).dt.to_period('M')
repay['Month']=pd.to_datetime(repay['Date']).dt.to_period('M')

monthly_spend = spend.groupby('Month')['Amount'].mean()
monthly_repay = repay.groupby('Month')['Amount'].mean()

monthly_profit = monthly_repay - monthly_spend
interest = monthly_profit.apply(lambda x: x*0.029 if x>0 else 0)

top_products = spend['Product'].value_counts().head(5)
city_max_spend = data.groupby('City')['Amount'].sum().idxmax()

acq['AgeGroup']=pd.cut(acq['Age'],bins=[0,30,50,100],labels=['Young','Middle','Senior'])
age_spend = data.groupby('AgeGroup')['Amount'].sum()

top10_repay = repay.groupby('Customer')['Amount'].sum().sort_values(ascending=False).head(10)

distinct_customers, distinct_categories, monthly_spend.head(), monthly_repay.head(), interest.head()


## 3. City-wise yearly spend by product

In [ ]:

spend['Year']=pd.to_datetime(spend['Date']).dt.year
city_year_prod = spend.groupby(['City','Year','Product'])['Amount'].sum().reset_index()
city_year_prod.head()


### Plot

In [ ]:

pivot = city_year_prod.pivot_table(index='City',columns='Product',values='Amount',aggfunc='sum')
pivot.plot(kind='bar')
plt.title("City-wise Spend by Product")
plt.show()


## 4. Monthly & Yearly Comparison Graphs

In [ ]:

# A – Monthly spend city-wise
city_month = spend.groupby(['City','Month'])['Amount'].sum().reset_index()
sns.lineplot(data=city_month,x='Month',y='Amount',hue='City')
plt.show()

# B – Yearly spend on air tickets
air = spend[spend['Category']=='AIR TICKET'].groupby('Year')['Amount'].sum()
air.plot(kind='bar'); plt.show()

# C – Monthly spend by product
prod_month = spend.groupby(['Product','Month'])['Amount'].sum().reset_index()
sns.lineplot(data=prod_month,x='Month',y='Amount',hue='Product')
plt.show()


## 5. Custom Function – Top 10 Customers by Product & Period

In [ ]:

def top10_customers(product, period):
    df = repay.merge(spend[['Customer','Product','Date']],on='Customer',how='left')
    df = df[df['Product']==product]
    df['Period'] = pd.to_datetime(df['Date']).dt.to_period('Y' if period=='year' else 'M')
    result = df.groupby(['City','Customer','Period'])['Amount_x'].sum().reset_index()
    return result.sort_values('Amount_x',ascending=False).head(10)

top10_customers('Gold','month')[:5]
